In [1]:
from numpy.random import choice
import numpy as np
import random

In [2]:
def crossOver(p1,p2):
    dadOrmom = [False, True]
    child = []

    cross = choice(dadOrmom, p=[0.5,0.5])

    if(cross):
        child.append(p1[0])
    else:
        child.append(p2[0])
    
    for i in range(1,10):
        cross = choice(dadOrmom, p=[0.5,0.5])
        if(cross):
            child.append(p1[i])
        else:
            child.append(p2[i])

    for i in range(10,18):
        cross = choice(dadOrmom, p=[0.5,0.5])

        if(cross):
            # if(i<=15):
            #     index = (i-9)*2+9
            #     child.append(p1[index-1])
            #     child.append(p1[index-0])
            # elif(i<=19):
            #     index = (i-15)*3+21
            #     child.append(p1[index-2])
            #     child.append(p1[index-1])
            #     child.append(p1[index-0])
            # else:
            index = (i-9)*4+9
            child.append(p1[index-3])
            child.append(p1[index-2])
            child.append(p1[index-1])
            child.append(p1[index-0])
        else:
            # if(i<=15):
            #     index = (i-9)*2+9
            #     child.append(p2[index-1])
            #     child.append(p2[index-0])
            # elif(i<=19):
            #     index = (i-15)*3+21
            #     child.append(p2[index-2])
            #     child.append(p2[index-1])
            #     child.append(p2[index-0])
            # else:
            index = (i-9)*4+9
            child.append(p2[index-3])
            child.append(p2[index-2])
            child.append(p2[index-1])
            child.append(p2[index-0])
    return child
    

In [3]:
def random2():
    p1 = np.random.uniform(low=0.0, high=1)
    p2 = 1-p1
    while(p1+p2!=1):
        p1 = np.random.uniform(low=0.0, high=1)
        p2 = 1-p1
    return p1,p2

def random3():
    p1 = np.random.uniform(low=0.0, high=1)
    p2 = np.random.uniform(low=0.0, high=(1-p1))
    p3 = 1-(p1+p2)
    while(p1+p2+p3!=1):
        p1 = np.random.uniform(low=0.0, high=1)
        p2 = np.random.uniform(low=0.0, high=(1-p1))
        p3 = 1-(p1+p2)
    return p1,p2,p3

def random4():
    p1 = np.random.uniform(low=0.0, high=1)
    p2 = np.random.uniform(low=0.0, high=(1-p1))
    p3 = np.random.uniform(low=0.0, high=(1-p1-p2))
    p4 = 1-(p1+p2+p3)
    while(p1+p2+p3+p4!=1):
        p1 = np.random.uniform(low=0.0, high=1)
        p2 = np.random.uniform(low=0.0, high=(1-p1))
        p3 = np.random.uniform(low=0.0, high=(1-p1-p2))
        p4 = 1-(p1+p2+p3)
    return p1,p2,p3,p4

In [4]:
def mutationChild(child):
    mutation = [False, True]

    ifMut = choice(mutation, p=[0.9,0.1])
    if(ifMut):
        child[0] = np.random.uniform(low=0.0, high=5.0)
    
    for i in range(1,10):
        ifMut = choice(mutation, p=[0.9,0.1])
        if(ifMut):
            if(i == 1):
                child[i] = random.randint(0, 30)
            else:
                child[i] = random.randint(0, 23)
    
    for i in range(10,18):
        ifMut = choice(mutation, p=[0.9,0.1])
        if(ifMut):
            # if(i<=15):
            #     index = (i-9)*2+9
            #     p1,p2 = random2()
            #     child[index-1] = p1
            #     child[index-0] = p2
            # elif(i<=19):
            #     index = (i-15)*3+21
            #     p1,p2,p3 = random3()
            #     child[index-2] = p1
            #     child[index-1] = p2
            #     child[index-0] = p3
            # else:
            index = (i-9)*4+9
            p1,p2,p3,p4 = random4()
            child[index-3] = p1
            child[index-2] = p2
            child[index-1] = p3
            child[index-0] = p4
    return child      

In [5]:
def writeString(child):
    string = ""
    for i in range(len(child)):
        string += str(child[i])+" "
    string+="\n"
    return string

In [6]:
import numpy as np
import random


string = "" + str(np.random.uniform(low=0.0, high=5.0))

string += " " + str(random.randint(0, 30))

for i in range(8):
    string += " " + str(random.randint(0, 23))

for i in range(8):
    # if(i<=15):
    #     p1,p2 = random2()
    #     string += " " + str(p1) + " " + str(p2)
    # elif(i<=19):
    #     p1,p2,p3 = random3()
    #     string += " " + str(p1) + " " + str(p2) + " " + str(p3)
    # else:
    p1,p2,p3,p4 = random4()
    string += " " + str(p1) + " " + str(p2) + " " + str(p3) + " " + str(p4)

with open('parameters.txt','w') as f:
    f.write(string)



for i in range(199):

    string = "\n" + str(np.random.uniform(low=0.0, high=5.0))

    string += " " + str(random.randint(0, 30))

    for i in range(8):
        string += " " + str(random.randint(0, 23))

    for i in range(8):
        # if(i<=15):
        #     p1,p2 = random2()
        #     string += " " + str(p1) + " " + str(p2)
        # elif(i<=19):
        #     p1,p2,p3 = random3()
        #     string += " " + str(p1) + " " + str(p2) + " " + str(p3)
        # else:
        p1,p2,p3,p4 = random4()
        string += " " + str(p1) + " " + str(p2) + " " + str(p3) + " " + str(p4)
                
    with open('parameters.txt','a') as f:
        f.write(string)

In [7]:
from numpy.random import choice
import numpy as np
import time

start_time = time.time()

population = 50
NotStop = True

generation = 0
with open('generation.txt','w') as f:
    f.write("")

with open('fitness.txt','w') as f:
    f.write("")

with open('Best.txt','w') as f:
    f.write("")

while(NotStop):

    # Run games

    with open('results.txt','w') as f:
            f.write("")
    for i in range(population):

        with open('agentNum.txt','w') as f:
            f.write(str(i))

        saida = !python pacman.py --layout originalClassic --pacman DumbAgent --numGames 2 --quietTextGraphics

    allPmt = []
    allRes = []
    allRes2 = []
    inds = []

    # Read parameters 
    for k in range (population):
        with open('parameters.txt') as f:
            lines = f.readlines()

            pmt = lines[k].split()

        for j in range(len(pmt)):
            if(1<=j<=9):
                pmt[j] = int(pmt[j])
            else:
                pmt[j] = float(pmt[j])
        
        allPmt.append(pmt)

    # Read results
    for k in range (population):
        with open('results.txt') as f:
            lines = f.readlines()

            res = lines[k].split()

        for j in range(len(res)):
            res[j] = float(res[j])
        
        allRes.append(res)


    # Select the best 5

    fitList = np.array([])

    for k in range (population):
        fit = allRes[k][1]+((229-allRes[k][3])/229)*2 + ((4-allRes[k][4])/4) + (min(allRes[k][5],1))
        fitList = np.append(fitList,fit)

    inds = np.argpartition(fitList, -5)[-5:]
    indsWorsts = np.argpartition(fitList, 5)[:5]

    print("Gen: ",generation)
    print("Best 5:")
    for i in inds:
        print(i,fitList[i],allRes[i])
    print("Worst 5:")
    for i in indsWorsts:   
        print(i,fitList[i],allRes[i])
    print()

    avgFit = sum(fitList)/len(fitList)
    avgBestFit = 0
    avgWorstFit = 0

    for i in inds:
        avgBestFit += fitList[i]
    avgBestFit = avgBestFit/5
    for i in indsWorsts:
        avgWorstFit+= fitList[i]
    avgWorstFit = avgWorstFit/5

    with open('fitness.txt','a') as f:
        s = str(avgBestFit) + " " + str(avgFit) + " " + str(avgWorstFit) + "\n"
        f.write(s)

    for i in inds:
        if(allRes[i][1]>=1):
            NotStop = False
            with open('Best.txt','a') as f:
                f.write(writeString(allPmt[i]))
                spentTime = (time.time() - start_time)
                f.write("Execution Time: "+str(spentTime))
    
    if(NotStop):

        with open('parameters.txt','w') as f:
            f.write("")


        # CrossOver

        childs = []

        for k in range(int(population/2)):
            d = choice(inds, p=[0.2,0.2,0.2,0.2,0.2])
            m = choice(inds, p=[0.2,0.2,0.2,0.2,0.2])
            while(d==m):
                m = choice(inds, p=[0.2,0.2,0.2,0.2,0.2])

            childs.append(crossOver(allPmt[d],allPmt[m]))

        # Mutation

        for k in range(int(population/2)):
            childs.append(mutationChild(childs[k]))

        with open('parameters.txt','a') as f:
            for k in range(population):
                f.write(writeString(childs[k]))

        

        with open('generation.txt','a') as f:
            f.write(str(generation)+"\n")
        generation+=1





Gen:  0
Best 5:
16 1.38264192139738 [-453.0, 0.0, 0.0, 199.5, 3.5, 1.50610041618]
27 1.4088427947598254 [-438.5, 0.0, 0.0, 196.5, 3.5, 1.19139742851]
42 1.4511283276905678 [-353.0, 0.0, 0.0, 191.5, 3.5, 0.99861741066]
12 1.5311135371179039 [-318.0, 0.0, 0.0, 182.5, 3.5, 1.19265055656]
38 1.8509825327510918 [-244.0, 0.0, 0.0, 174.5, 2.5, 1.26216614246]
Worst 5:
5 0.25365051744274236 [-496.0, 0.0, 0.0, 224.0, 4.0, 0.209982395172]
14 0.32531982321963315 [-439.5, 0.0, 0.0, 218.0, 4.0, 0.229249954224]
20 0.27038306804733625 [-451.0, 0.0, 0.0, 220.0, 4.0, 0.19178044796]
22 0.32392784497589083 [-500.0, 0.0, 0.0, 223.0, 4.0, 0.271526098251]
40 0.2518255481555939 [-508.0, 0.0, 0.0, 225.0, 4.0, 0.216891050339]

Gen:  1
Best 5:
3 1.5731441048034935 [-282.0, 0.0, 0.0, 192.0, 3.0, 1.51431643963]
11 1.5791484716157205 [-380.5, 0.0, 0.0, 177.0, 3.5, 1.68018496037]
8 1.8466157205240175 [-180.5, 0.0, 0.0, 175.0, 2.5, 1.39362859726]
22 2.0998908296943233 [35.0, 0.0, 0.0, 146.0, 2.5, 1.65281403065]
47 1.

In [8]:
!python pacman.py --help

Usage: 
    USAGE:      python pacman.py <options>
    EXAMPLES:   (1) python pacman.py
                    - starts an interactive game
                (2) python pacman.py --layout smallClassic --zoom 2
                OR  python pacman.py -l smallClassic -z 2
                    - starts an interactive game on a smaller board, zoomed in
    

Options:
  -h, --help            show this help message and exit
  -n GAMES, --numGames=GAMES
                        the number of GAMES to play [Default: 1]
  -l LAYOUT_FILE, --layout=LAYOUT_FILE
                        the LAYOUT_FILE from which to load the map layout
                        [Default: mediumClassic]
  -p TYPE, --pacman=TYPE
                        the agent TYPE in the pacmanAgents module to use
                        [Default: KeyboardAgent]
  -t, --textGraphics    Display output as text only
  -q, --quietTextGraphics
                        Generate minimal output and no graphics
  -g TYPE, --ghosts=TYPE
                 

In [2]:
!python pacman.py --layout originalClassic --pacman DumbAgent --numGames 8

Pacman died! Score: -337
Pacman died! Score: -332
Pacman died! Score: -417
Pacman died! Score: -257
Pacman died! Score: -394
Pacman died! Score: -298
Pacman died! Score: -365
Pacman died! Score: -486
Average Score: -360.75
Scores:        -337.0, -332.0, -417.0, -257.0, -394.0, -298.0, -365.0, -486.0
Win Rate:      0/8 (0.00)
Record:        Loss, Loss, Loss, Loss, Loss, Loss, Loss, Loss
